In [31]:
import pandas as pd
import numpy as np

from dsgd.DSClassifierMultiQ import DSClassifierMultiQ



from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Clasificadores
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Exploracion de Datos y pre procesamiento

In [32]:
data = pd.read_csv('data/list_attr_celeba.csv')
print("tamaño del dataset: ", data.shape)
data = data.drop('image_id', axis=1)
data = data.replace(-1, 0)
data = data.sample(frac=1).reset_index(drop=True)
print(data.dtypes)

tamaño del dataset:  (202599, 41)
5_o_Clock_Shadow       int64
Arched_Eyebrows        int64
Attractive             int64
Bags_Under_Eyes        int64
Bald                   int64
Bangs                  int64
Big_Lips               int64
Big_Nose               int64
Black_Hair             int64
Blond_Hair             int64
Blurry                 int64
Brown_Hair             int64
Bushy_Eyebrows         int64
Chubby                 int64
Double_Chin            int64
Eyeglasses             int64
Goatee                 int64
Gray_Hair              int64
Heavy_Makeup           int64
High_Cheekbones        int64
Male                   int64
Mouth_Slightly_Open    int64
Mustache               int64
Narrow_Eyes            int64
No_Beard               int64
Oval_Face              int64
Pale_Skin              int64
Pointy_Nose            int64
Receding_Hairline      int64
Rosy_Cheeks            int64
Sideburns              int64
Smiling                int64
Straight_Hair          int64
Wavy_Hair

In [33]:
y = data["Attractive"]
X = data.drop(columns=["Attractive"])

# Testeo de los clasificadores

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [35]:
# Diccionario de modelos
accuracys = {}
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(max_depth=5),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

# Función para entrenar y evaluar modelos
def train_and_evaluate(models, X_train, y_train, X_test, y_test):
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'{name} Accuracy: {accuracy:.4f}')
        accuracys[name] = accuracy
        print(f'Classification Report for {name}:\n{classification_report(y_test, y_pred)}')
        print(f'Confusion Matrix for {name}:\n{confusion_matrix(y_test, y_pred)}\n')
        print(f'---------------------------------------------------\n')

# Llamada a la función
train_and_evaluate(models, X_train, y_train, X_test, y_test)


Logistic Regression Accuracy: 0.7843
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.79      0.76      0.77     24692
           1       0.78      0.81      0.79     25958

    accuracy                           0.78     50650
   macro avg       0.78      0.78      0.78     50650
weighted avg       0.78      0.78      0.78     50650

Confusion Matrix for Logistic Regression:
[[18767  5925]
 [ 4998 20960]]

---------------------------------------------------

Random Forest Accuracy: 0.7564
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.72      0.81      0.76     24692
           1       0.79      0.71      0.75     25958

    accuracy                           0.76     50650
   macro avg       0.76      0.76      0.76     50650
weighted avg       0.76      0.76      0.76     50650

Confusion Matrix for Random Forest:
[[19907  4785]
 [ 7555 1840

# Testeo del clasificador de DS

In [36]:
Y = y
X = X.apply(pd.to_numeric)

cut = int(0.25 * len(data))

X_train = X.iloc[:-cut].values
X_test = X.iloc[-cut:].values
Y_train = Y.iloc[:-cut].values
Y_test = Y.iloc[-cut:].values


print(len(X_train), len(X_test), len(Y_train), len(Y_test))
print(Y.head())


151950 50649 151950 50649
0    0
1    1
2    1
3    0
4    0
Name: Attractive, dtype: int64


## clasificador 1

In [37]:
DSC = DSClassifierMultiQ(2, min_iter=50, max_iter=400, debug_mode=True, \
                         lossfn="MSE", num_workers=0, min_dloss=1e-7, precompute_rules=True)

In [38]:
losses, epoch, dt = DSC.fit(X_train, Y_train, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=False,
                                column_names=data.columns[:-1], print_every_epochs=31, print_final_model=True)

Optimization started
Processing epoch	32	0.1453	
Training time: 1781.95s, epochs: 52

Least training loss reached: 0.145
DSModelMultiQ(
  DS Classifier using 78 rules
  
  Rule 1: 5_o_Clock_Shadow = 0
  	C1: 0.165	C2: 0.000	Unc: 0.835
  
  Rule 2: 5_o_Clock_Shadow = 1
  	C1: 0.000	C2: 0.257	Unc: 0.743
  
  Rule 3: Arched_Eyebrows = 0
  	C1: 0.092	C2: 0.063	Unc: 0.845
  
  Rule 4: Arched_Eyebrows = 1
  	C1: 0.044	C2: 0.169	Unc: 0.786
  
  Rule 5: Attractive = 0
  	C1: 0.096	C2: 0.027	Unc: 0.877
  
  Rule 6: Attractive = 1
  	C1: 0.089	C2: 0.098	Unc: 0.813
  
  Rule 7: Bags_Under_Eyes = 0
  	C1: 0.055	C2: 0.088	Unc: 0.858
  
  Rule 8: Bags_Under_Eyes = 1
  	C1: 0.725	C2: 0.000	Unc: 0.275
  
  Rule 9: Bald = 0
  	C1: 0.059	C2: 0.068	Unc: 0.873
  
  Rule 10: Bald = 1
  	C1: 0.126	C2: 0.011	Unc: 0.863
  
  Rule 11: Bangs = 0
  	C1: 0.093	C2: 0.034	Unc: 0.873
  
  Rule 12: Bangs = 1
  	C1: 0.122	C2: 0.004	Unc: 0.874
  
  Rule 13: Big_Lips = 0
  	C1: 0.004	C2: 0.199	Unc: 0.797
  
  Rule 14: B

In [39]:
DSC.print_most_important_rules(classes=["bad appearance",  "good appearance"])



Most important rules for class bad appearance

	[0.809] R25: Bushy_Eyebrows = 1
			bad: 0.809	goo: 0.000	Unc: 0.191

	[0.725] R7: Bags_Under_Eyes = 1
			bad: 0.725	goo: 0.000	Unc: 0.275

	[0.714] R19: Blond_Hair = 1
			bad: 0.713	goo: 0.002	Unc: 0.285

	[0.659] R33: Goatee = 1
			bad: 0.656	goo: 0.005	Unc: 0.339

	[0.640] R27: Chubby = 1
			bad: 0.638	goo: 0.004	Unc: 0.358

	[0.565] R76: Wearing_Necktie = 0
			bad: 0.565	goo: 0.001	Unc: 0.434

	[0.551] R29: Double_Chin = 1
			bad: 0.551	goo: 0.001	Unc: 0.448

	[0.470] R55: Pointy_Nose = 1
			bad: 0.470	goo: 0.000	Unc: 0.530

	[0.416] R70: Wearing_Hat = 0
			bad: 0.416	goo: 0.000	Unc: 0.584

	[0.391] R34: Gray_Hair = 0
			bad: 0.390	goo: 0.001	Unc: 0.609

	[0.352] R69: Wearing_Earrings = 1
			bad: 0.352	goo: 0.002	Unc: 0.647

	[0.338] R60: Sideburns = 0
			bad: 0.338	goo: 0.000	Unc: 0.662

	[0.291] R45: Mustache = 1
			bad: 0.286	goo: 0.009	Unc: 0.704

	[0.241] R22: Brown_Hair = 0
			bad: 0.239	goo: 0.003	Unc: 0.757

	[0.204] R67: Wav

In [40]:
Y_pred = DSC.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
accuracys["DSClassifierMultiQ"] = accuracy
print(f'Accuracy: {accuracy:.4f}')
print(f'Classification Report:\n{classification_report(Y_test, Y_pred)}')
print(f'Confusion Matrix:\n{confusion_matrix(Y_test, Y_pred)}\n')

Accuracy: 0.7897
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.76      0.78     24750
           1       0.78      0.82      0.80     25899

    accuracy                           0.79     50649
   macro avg       0.79      0.79      0.79     50649
weighted avg       0.79      0.79      0.79     50649

Confusion Matrix:
[[18733  6017]
 [ 4636 21263]]



## clasifificador 2

In [41]:
DSC2 = DSClassifierMultiQ(2, min_iter=50, max_iter=400, debug_mode=True, optim="sgd", \
                         lossfn="MSE", num_workers=0, min_dloss=1e-7, precompute_rules=True)

In [42]:
losses, epoch, dt = DSC2.fit(X_train, Y_train, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=False,
                                column_names=data.columns[:-1], print_every_epochs=31, print_final_model=True)

Optimization started
Processing epoch	373	0.1482	
Training time: 13585.41s, epochs: 400

Least training loss reached: 0.148
DSModelMultiQ(
  DS Classifier using 78 rules
  
  Rule 1: 5_o_Clock_Shadow = 0
  	C1: 0.211	C2: 0.000	Unc: 0.789
  
  Rule 2: 5_o_Clock_Shadow = 1
  	C1: 0.010	C2: 0.182	Unc: 0.808
  
  Rule 3: Arched_Eyebrows = 0
  	C1: 0.139	C2: 0.056	Unc: 0.805
  
  Rule 4: Arched_Eyebrows = 1
  	C1: 0.043	C2: 0.153	Unc: 0.804
  
  Rule 5: Attractive = 0
  	C1: 0.128	C2: 0.070	Unc: 0.802
  
  Rule 6: Attractive = 1
  	C1: 0.090	C2: 0.109	Unc: 0.801
  
  Rule 7: Bags_Under_Eyes = 0
  	C1: 0.086	C2: 0.113	Unc: 0.801
  
  Rule 8: Bags_Under_Eyes = 1
  	C1: 0.155	C2: 0.041	Unc: 0.804
  
  Rule 9: Bald = 0
  	C1: 0.131	C2: 0.066	Unc: 0.803
  
  Rule 10: Bald = 1
  	C1: 0.171	C2: 0.027	Unc: 0.802
  
  Rule 11: Bangs = 0
  	C1: 0.122	C2: 0.076	Unc: 0.802
  
  Rule 12: Bangs = 1
  	C1: 0.152	C2: 0.045	Unc: 0.804
  
  Rule 13: Big_Lips = 0
  	C1: 0.000	C2: 0.205	Unc: 0.795
  
  Rule 14

In [43]:
DSC2.print_most_important_rules(classes=["bad appearance",  "good appearance"])



Most important rules for class bad appearance

	[0.470] R76: Wearing_Necktie = 0
			bad: 0.470	goo: 0.000	Unc: 0.530

	[0.387] R34: Gray_Hair = 0
			bad: 0.387	goo: 0.000	Unc: 0.613

	[0.320] R70: Wearing_Hat = 0
			bad: 0.320	goo: 0.000	Unc: 0.680

	[0.302] R29: Double_Chin = 1
			bad: 0.302	goo: 0.000	Unc: 0.698

	[0.297] R19: Blond_Hair = 1
			bad: 0.297	goo: 0.000	Unc: 0.703

	[0.276] R22: Brown_Hair = 0
			bad: 0.276	goo: 0.000	Unc: 0.724

	[0.274] R60: Sideburns = 0
			bad: 0.274	goo: 0.000	Unc: 0.726

	[0.252] R25: Bushy_Eyebrows = 1
			bad: 0.252	goo: 0.000	Unc: 0.748

	[0.226] R55: Pointy_Nose = 1
			bad: 0.226	goo: 0.000	Unc: 0.774

	[0.211] R0: 5_o_Clock_Shadow = 0
			bad: 0.211	goo: 0.000	Unc: 0.789

	[0.210] R48: No_Beard = 0
			bad: 0.210	goo: 0.000	Unc: 0.790

	[0.209] R62: Smiling = 0
			bad: 0.209	goo: 0.000	Unc: 0.791

	[0.208] R50: Oval_Face = 0
			bad: 0.208	goo: 0.000	Unc: 0.792

	[0.207] R52: Pale_Skin = 0
			bad: 0.207	goo: 0.000	Unc: 0.793

	[0.205] R13: Big_L

In [44]:
Y_pred2 = DSC2.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred2)
accuracys["DSClassifierMultiQ SGD"] = accuracy
print(f'Accuracy: {accuracy:.4f}')
print(f'Classification Report:\n{classification_report(Y_test, Y_pred2)}')
print(f'Confusion Matrix:\n{confusion_matrix(Y_test, Y_pred2)}\n')

Accuracy: 0.7841
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.77      0.78     24750
           1       0.78      0.80      0.79     25899

    accuracy                           0.78     50649
   macro avg       0.78      0.78      0.78     50649
weighted avg       0.78      0.78      0.78     50649

Confusion Matrix:
[[18994  5756]
 [ 5179 20720]]



## clasificador 3

In [45]:
DSC3 = DSClassifierMultiQ(2, min_iter=50, max_iter=400, debug_mode=True, \
                         lossfn="CE", num_workers=0, min_dloss=1e-7, precompute_rules=True)

In [46]:
losses, epoch, dt = DSC3.fit(X_train, Y_train, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=False,
                                column_names=data.columns[:-1], print_every_epochs=31, print_final_model=True)

Optimization started
Processing epoch	32	0.5119	
Training time: 1954.08s, epochs: 54

Least training loss reached: 0.512
DSModelMultiQ(
  DS Classifier using 78 rules
  
  Rule 1: 5_o_Clock_Shadow = 0
  	C1: 0.321	C2: 0.003	Unc: 0.676
  
  Rule 2: 5_o_Clock_Shadow = 1
  	C1: 0.000	C2: 0.474	Unc: 0.525
  
  Rule 3: Arched_Eyebrows = 0
  	C1: 0.181	C2: 0.004	Unc: 0.816
  
  Rule 4: Arched_Eyebrows = 1
  	C1: 0.032	C2: 0.217	Unc: 0.752
  
  Rule 5: Attractive = 0
  	C1: 0.113	C2: 0.014	Unc: 0.872
  
  Rule 6: Attractive = 1
  	C1: 0.120	C2: 0.094	Unc: 0.785
  
  Rule 7: Bags_Under_Eyes = 0
  	C1: 0.001	C2: 0.115	Unc: 0.884
  
  Rule 8: Bags_Under_Eyes = 1
  	C1: 0.984	C2: 0.000	Unc: 0.016
  
  Rule 9: Bald = 0
  	C1: 0.084	C2: 0.064	Unc: 0.852
  
  Rule 10: Bald = 1
  	C1: 0.086	C2: 0.021	Unc: 0.893
  
  Rule 11: Bangs = 0
  	C1: 0.059	C2: 0.074	Unc: 0.868
  
  Rule 12: Bangs = 1
  	C1: 0.160	C2: 0.005	Unc: 0.835
  
  Rule 13: Big_Lips = 0
  	C1: 0.001	C2: 0.383	Unc: 0.616
  
  Rule 14: B

In [47]:
DSC3.print_most_important_rules(classes=["bad appearance",  "good appearance"])



Most important rules for class bad appearance

	[0.985] R19: Blond_Hair = 1
			bad: 0.983	goo: 0.004	Unc: 0.013

	[0.984] R7: Bags_Under_Eyes = 1
			bad: 0.984	goo: 0.000	Unc: 0.016

	[0.969] R76: Wearing_Necktie = 0
			bad: 0.969	goo: 0.000	Unc: 0.031

	[0.960] R27: Chubby = 1
			bad: 0.960	goo: 0.000	Unc: 0.040

	[0.952] R25: Bushy_Eyebrows = 1
			bad: 0.937	goo: 0.031	Unc: 0.032

	[0.948] R33: Goatee = 1
			bad: 0.948	goo: 0.001	Unc: 0.051

	[0.904] R29: Double_Chin = 1
			bad: 0.903	goo: 0.001	Unc: 0.096

	[0.880] R55: Pointy_Nose = 1
			bad: 0.880	goo: 0.000	Unc: 0.120

	[0.829] R70: Wearing_Hat = 0
			bad: 0.829	goo: 0.000	Unc: 0.171

	[0.701] R69: Wearing_Earrings = 1
			bad: 0.699	goo: 0.004	Unc: 0.297

	[0.676] R60: Sideburns = 0
			bad: 0.676	goo: 0.000	Unc: 0.324

	[0.560] R34: Gray_Hair = 0
			bad: 0.559	goo: 0.000	Unc: 0.440

	[0.523] R45: Mustache = 1
			bad: 0.519	goo: 0.007	Unc: 0.474

	[0.497] R67: Wavy_Hair = 1
			bad: 0.497	goo: 0.000	Unc: 0.503

	[0.470] R41: Male

In [48]:
Y_pred3 = DSC3.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred3)
accuracys["DSClassifierMultiQ CE"] = accuracy
print(f'Accuracy: {accuracy:.4f}')
print(f'Classification Report:\n{classification_report(Y_test, Y_pred3)}')
print(f'Confusion Matrix:\n{confusion_matrix(Y_test, Y_pred3)}\n')

Accuracy: 0.7896
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.75      0.78     24750
           1       0.78      0.83      0.80     25899

    accuracy                           0.79     50649
   macro avg       0.79      0.79      0.79     50649
weighted avg       0.79      0.79      0.79     50649

Confusion Matrix:
[[18548  6202]
 [ 4457 21442]]



## clasifificador 4

In [49]:
DSC4 = DSClassifierMultiQ(2, min_iter=50, max_iter=400, debug_mode=True, optim="sgd", \
                         lossfn="CE", num_workers=0, min_dloss=1e-7, precompute_rules=True)

In [50]:
losses, epoch, dt = DSC4.fit(X_train, Y_train, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=False,
                                column_names=data.columns[:-1], print_every_epochs=31, print_final_model=True)

Optimization started
Processing epoch	373	0.5272	
Training time: 14248.44s, epochs: 400

Least training loss reached: 0.526
DSModelMultiQ(
  DS Classifier using 78 rules
  
  Rule 1: 5_o_Clock_Shadow = 0
  	C1: 0.255	C2: 0.000	Unc: 0.745
  
  Rule 2: 5_o_Clock_Shadow = 1
  	C1: 0.000	C2: 0.242	Unc: 0.758
  
  Rule 3: Arched_Eyebrows = 0
  	C1: 0.188	C2: 0.000	Unc: 0.812
  
  Rule 4: Arched_Eyebrows = 1
  	C1: 0.008	C2: 0.244	Unc: 0.749
  
  Rule 5: Attractive = 0
  	C1: 0.118	C2: 0.080	Unc: 0.802
  
  Rule 6: Attractive = 1
  	C1: 0.111	C2: 0.088	Unc: 0.801
  
  Rule 7: Bags_Under_Eyes = 0
  	C1: 0.134	C2: 0.065	Unc: 0.801
  
  Rule 8: Bags_Under_Eyes = 1
  	C1: 0.091	C2: 0.115	Unc: 0.795
  
  Rule 9: Bald = 0
  	C1: 0.081	C2: 0.119	Unc: 0.800
  
  Rule 10: Bald = 1
  	C1: 0.105	C2: 0.095	Unc: 0.800
  
  Rule 11: Bangs = 0
  	C1: 0.103	C2: 0.096	Unc: 0.801
  
  Rule 12: Bangs = 1
  	C1: 0.150	C2: 0.047	Unc: 0.803
  
  Rule 13: Big_Lips = 0
  	C1: 0.000	C2: 0.293	Unc: 0.707
  
  Rule 14

In [51]:
DSC4.print_most_important_rules(classes=["bad appearance",  "good appearance"])



Most important rules for class bad appearance

	[0.651] R76: Wearing_Necktie = 0
			bad: 0.651	goo: 0.000	Unc: 0.349

	[0.538] R70: Wearing_Hat = 0
			bad: 0.538	goo: 0.000	Unc: 0.462

	[0.506] R34: Gray_Hair = 0
			bad: 0.506	goo: 0.000	Unc: 0.494

	[0.382] R60: Sideburns = 0
			bad: 0.382	goo: 0.000	Unc: 0.618

	[0.374] R52: Pale_Skin = 0
			bad: 0.374	goo: 0.000	Unc: 0.626

	[0.372] R13: Big_Lips = 1
			bad: 0.372	goo: 0.000	Unc: 0.628

	[0.365] R48: No_Beard = 0
			bad: 0.365	goo: 0.000	Unc: 0.635

	[0.348] R22: Brown_Hair = 0
			bad: 0.348	goo: 0.000	Unc: 0.652

	[0.315] R55: Pointy_Nose = 1
			bad: 0.315	goo: 0.000	Unc: 0.685

	[0.310] R64: Straight_Hair = 0
			bad: 0.310	goo: 0.000	Unc: 0.690

	[0.306] R19: Blond_Hair = 1
			bad: 0.306	goo: 0.000	Unc: 0.694

	[0.277] R62: Smiling = 0
			bad: 0.277	goo: 0.000	Unc: 0.723

	[0.272] R33: Goatee = 1
			bad: 0.272	goo: 0.000	Unc: 0.728

	[0.261] R20: Blurry = 0
			bad: 0.261	goo: 0.000	Unc: 0.739

	[0.255] R0: 5_o_Clock_Shadow = 0
	

In [52]:
Y_pred4 = DSC4.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred4)
accuracys["DSClassifierMultiQ CE  SGD"] = accuracy
print(f'Accuracy: {accuracy:.4f}')
print(f'Classification Report:\n{classification_report(Y_test, Y_pred4)}')
print(f'Confusion Matrix:\n{confusion_matrix(Y_test, Y_pred4)}\n')

Accuracy: 0.7805
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.77      0.77     24750
           1       0.78      0.79      0.79     25899

    accuracy                           0.78     50649
   macro avg       0.78      0.78      0.78     50649
weighted avg       0.78      0.78      0.78     50649

Confusion Matrix:
[[19023  5727]
 [ 5392 20507]]



## clasifificador 5

In [53]:
DSC5 = DSClassifierMultiQ(2, min_iter=50, max_iter=400, debug_mode=True, \
                         lossfn="MSE", num_workers=0, min_dloss=1e-7, precompute_rules=True)

In [54]:
losses, epoch, dt = DSC5.fit(X_train, Y_train, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=True,
                                column_names=data.columns[:-1], print_every_epochs=31, print_final_model=True)

Optimization started
Processing epoch	32	0.2500	
Training time: 3334.06s, epochs: 52

Least training loss reached: 0.250
DSModelMultiQ(
  DS Classifier using 1560 rules
  
  Rule 1: 5_o_Clock_Shadow = 0
  	C1: 0.124	C2: 0.076	Unc: 0.800
  
  Rule 2: 5_o_Clock_Shadow = 1
  	C1: 0.062	C2: 0.138	Unc: 0.800
  
  Rule 3: Arched_Eyebrows = 0
  	C1: 0.147	C2: 0.053	Unc: 0.800
  
  Rule 4: Arched_Eyebrows = 1
  	C1: 0.108	C2: 0.092	Unc: 0.800
  
  Rule 5: Attractive = 0
  	C1: 0.046	C2: 0.154	Unc: 0.800
  
  Rule 6: Attractive = 1
  	C1: 0.080	C2: 0.120	Unc: 0.800
  
  Rule 7: Bags_Under_Eyes = 0
  	C1: 0.143	C2: 0.057	Unc: 0.800
  
  Rule 8: Bags_Under_Eyes = 1
  	C1: 0.137	C2: 0.063	Unc: 0.800
  
  Rule 9: Bald = 0
  	C1: 0.153	C2: 0.047	Unc: 0.800
  
  Rule 10: Bald = 1
  	C1: 0.108	C2: 0.092	Unc: 0.800
  
  Rule 11: Bangs = 0
  	C1: 0.092	C2: 0.108	Unc: 0.800
  
  Rule 12: Bangs = 1
  	C1: 0.095	C2: 0.105	Unc: 0.800
  
  Rule 13: Big_Lips = 0
  	C1: 0.107	C2: 0.093	Unc: 0.800
  
  Rule 14:

In [58]:
DSC5.print_most_important_rules(classes=["bad appearance",  "good appearance"])



Most important rules for class bad appearance

Most important rules for class good appearance


In [56]:
Y_pred5 = DSC5.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred5)
accuracys["DSClassifierMultiQ MSE adam multi-layer"] = accuracy
print(f'Accuracy: {accuracy:.4f}')
print(f'Classification Report:\n{classification_report(Y_test, Y_pred5)}')
print(f'Confusion Matrix:\n{confusion_matrix(Y_test, Y_pred5)}\n')

Accuracy: 0.4887
Classification Report:
              precision    recall  f1-score   support

           0       0.49      1.00      0.66     24750
           1       0.00      0.00      0.00     25899

    accuracy                           0.49     50649
   macro avg       0.24      0.50      0.33     50649
weighted avg       0.24      0.49      0.32     50649

Confusion Matrix:
[[24750     0]
 [25899     0]]



c:\Users\Daniel\Documents\GitHub\trabajo-dirigido\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Daniel\Documents\GitHub\trabajo-dirigido\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Daniel\Documents\GitHub\trabajo-dirigido\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

## resultados

In [57]:
for key in accuracys:
    print(key, accuracys[key])

Logistic Regression 0.7843435340572557
Random Forest 0.7563672260612043
Support Vector Machine 0.7904442250740376
K-Nearest Neighbors 0.7599012833168806
DSClassifierMultiQ 0.7896700823313392
DSClassifierMultiQ SGD 0.7841023514778179
DSClassifierMultiQ CE 0.7895516199727537
DSClassifierMultiQ CE  SGD 0.7804695058145275
DSClassifierMultiQ MSE adam multi-layer 0.4886572291654327
